# 🌿 PHÂN LOẠI VẾT BỆNH TRÊN LÁ ĐẬU (Bean Leaf Lesions Classification)

## Bài Tập Lớn - Môn Khai Phá Dữ Liệu

---

### 📋 Mục Lục

1. **Giới thiệu bài toán**
2. **Nhập thư viện & tải dữ liệu**
3. **Khám phá dữ liệu (EDA)**
4. **Tiền xử lý ảnh**
5. **Xây dựng mô hình Deep Learning**
6. **Đánh giá & phân tích kết quả**
7. **So sánh mô hình và kết luận**
8. **Đề xuất nâng cao**

---

### 🎯 Giới thiệu bài toán

**Dataset:** Bean Leaf Lesions Classification từ Kaggle

**Mô tả:** Dataset gồm ảnh lá đậu thuộc 3 trạng thái:
- 🟢 **Healthy** (lá khỏe)
- 🟡 **Angular Leaf Spot** (vết bệnh góc)
- 🔴 **Bean Rust** (bệnh rỉ sắt)

**Nguồn dữ liệu:** [Kaggle - Bean Leaf Lesions Classification](https://www.kaggle.com/datasets/marquis03/bean-leaf-lesions-classification)

**Mục tiêu:** Xây dựng các mô hình Deep Learning để phân loại chính xác các trạng thái bệnh trên lá đậu, giúp hỗ trợ nông nghiệp thông minh và phát hiện bệnh sớm cho cây trồng.

---

### 👥 Thông tin nhóm & Phân công nhiệm vụ

| STT | Họ và Tên | MSSV | Vai trò | Mô hình phụ trách |
|-----|-----------|------|---------|-------------------|
| 1 | [Thành viên 1] | [MSSV] | Nhóm trưởng | ResNet50 |
| 2 | [Thành viên 2] | [MSSV] | Thành viên | MobileNetV2 |
| 3 | [Thành viên 3] | [MSSV] | Thành viên | VGG19 |

**Phân công nhiệm vụ cụ thể:**
- **[Thành viên 1]:** Thu thập dữ liệu, EDA, huấn luyện ResNet50, viết báo cáo
- **[Thành viên 2]:** Data Augmentation, huấn luyện MobileNetV2, đánh giá kết quả
- **[Thành viên 3]:** Thiết kế mô hình, huấn luyện VGG19, tối ưu hyperparameters

> **Lưu ý:** Mỗi thành viên đã áp dụng ít nhất 1 mô hình học máy theo yêu cầu đồ án.

---

### 👨‍💻 Thông tin kỹ thuật

- **Ngày tạo:** 2024
- **Framework:** TensorFlow/Keras
- **Ngôn ngữ:** Python 3.10+

---

## 1️⃣ Nhập Thư Viện & Thiết Lập Môi Trường

### Các thư viện sử dụng:
- **numpy, pandas:** Xử lý dữ liệu số và bảng
- **matplotlib, seaborn:** Trực quan hóa dữ liệu
- **TensorFlow/Keras:** Xây dựng và huấn luyện mô hình Deep Learning
- **scikit-learn:** Đánh giá và phân loại

In [ ]:
# ============================================
# NHẬP CÁC THƯ VIỆN CẦN THIẾT
# ============================================

# Thư viện xử lý dữ liệu
import numpy as np
import pandas as pd
import os
import random
from pathlib import Path
from collections import Counter

# Thư viện trực quan hóa
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình matplotlib cho tiếng Việt
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# Thư viện Deep Learning - TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, MobileNetV2, VGG19
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Thư viện đánh giá mô hình
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Thư viện xử lý ảnh
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Kiểm tra phiên bản TensorFlow và GPU
print("=" * 60)
print("THÔNG TIN MÔI TRƯỜNG")
print("=" * 60)
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Thiết lập seed để đảm bảo kết quả có thể tái tạo
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
print(f"
Random seed đã được thiết lập: {SEED}")
print("=" * 60)

### 📂 Tải và kiểm tra cấu trúc dữ liệu

Bước này kiểm tra cấu trúc thư mục ảnh và các nhãn lớp (classes) trong dataset.

In [ ]:
# ============================================
# KIỂM TRA CẤU TRÚC DỮ LIỆU
# ============================================

# Đường dẫn đến thư mục dữ liệu
DATA_DIR = './data'

# Kiểm tra xem thư mục data có tồn tại không
if not os.path.exists(DATA_DIR):
    print("Vui lòng chạy: python download_dataset.py")
else:
    print("✅ Đã tìm thấy thư mục data")
    
    # Liệt kê các thư mục con
    print("
📁 Cấu trúc thư mục dữ liệu:")
    for item in sorted(os.listdir(DATA_DIR)):
        item_path = os.path.join(DATA_DIR, item)
        if os.path.isdir(item_path):
            sub_items = os.listdir(item_path)
            print(f"  📂 {item}/ ({len(sub_items)} mục)")
            # Hiển thị một số mục con
            for sub in sorted(sub_items)[:5]:
                sub_path = os.path.join(item_path, sub)
                if os.path.isdir(sub_path):
                    files_in_sub = len(os.listdir(sub_path))
                    print(f"      📂 {sub}/ ({files_in_sub} ảnh)")
        elif os.path.isfile(item_path) and item != 'README.md':
            size_mb = os.path.getsize(item_path) / (1024 * 1024)
            print(f"  📄 {item} ({size_mb:.2f} MB)")

In [ ]:
# ============================================
# THIẾT LẬP ĐƯỜNG DẪN DỮ LIỆU
# ============================================

# Đường dẫn đến các tập dữ liệu
# Cấu trúc thường gặp của dataset này:
# data/
#   ├── train/
#   │   ├── angular_leaf_spot/
#   │   ├── bean_rust/
#   │   └── healthy/
#   └── validation/ (hoặc test/)
#       ├── angular_leaf_spot/
#       ├── bean_rust/
#       └── healthy/

# Tự động phát hiện cấu trúc thư mục
def find_data_directories(base_path):
    """Tìm các thư mục train và validation trong dataset"""
    train_dir = None
    val_dir = None
    
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            item_lower = item.lower()
            if 'train' in item_lower:
                train_dir = item_path
            elif 'val' in item_lower or 'test' in item_lower:
                val_dir = item_path
    
    return train_dir, val_dir

TRAIN_DIR, VAL_DIR = find_data_directories(DATA_DIR)

print("📁 Đường dẫn dữ liệu:")
print(f"   Train: {TRAIN_DIR}")
print(f"   Validation/Test: {VAL_DIR}")

# Nếu không tìm thấy, sử dụng đường dẫn mặc định
if TRAIN_DIR is None:
    TRAIN_DIR = os.path.join(DATA_DIR, 'train')
    print(f"
⚠️ Sử dụng đường dẫn mặc định cho train: {TRAIN_DIR}")
    
if VAL_DIR is None:
    VAL_DIR = os.path.join(DATA_DIR, 'validation')
    print(f"⚠️ Sử dụng đường dẫn mặc định cho validation: {VAL_DIR}")

# Lấy danh sách các lớp (classes)
if os.path.exists(TRAIN_DIR):
    CLASSES = sorted([d for d in os.listdir(TRAIN_DIR) 
                      if os.path.isdir(os.path.join(TRAIN_DIR, d))])
    print(f"
🏷️ Các lớp phân loại: {CLASSES}")
    print(f"   Số lượng lớp: {len(CLASSES)}")
else:
    CLASSES = ['angular_leaf_spot', 'bean_rust', 'healthy']
    print(f"
⚠️ Sử dụng danh sách lớp mặc định: {CLASSES}")

---

## 2️⃣ Khám Phá Dữ Liệu (EDA - Exploratory Data Analysis)

Trong phần này, chúng ta sẽ:
- Đếm số lượng ảnh theo từng lớp
- Phân tích kích thước ảnh
- Hiển thị ví dụ ảnh trực quan
- Vẽ biểu đồ phân bố dữ liệu

In [ ]:
# ============================================
# ĐẾM SỐ LƯỢNG ẢNH THEO TỪNG LỚP
# ============================================

def count_images_by_class(data_dir):
    """Đếm số lượng ảnh trong mỗi lớp của thư mục dữ liệu"""
    class_counts = {}
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    
    if not os.path.exists(data_dir):
        return class_counts
    
    for class_name in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):
            count = 0
            for file in os.listdir(class_path):
                if any(file.lower().endswith(ext) for ext in image_extensions):
                    count += 1
            class_counts[class_name] = count
    
    return class_counts

# Đếm ảnh trong tập train
train_counts = count_images_by_class(TRAIN_DIR)
print("📊 SỐ LƯỢNG ẢNH TRONG TẬP TRAIN:")
print("=" * 40)
total_train = 0
for class_name, count in sorted(train_counts.items()):
    print(f"   {class_name}: {count} ảnh")
    total_train += count
print(f"   ─────────────────────────")
print(f"   Tổng cộng: {total_train} ảnh")

# Đếm ảnh trong tập validation/test
val_counts = count_images_by_class(VAL_DIR)
print("
📊 SỐ LƯỢNG ẢNH TRONG TẬP VALIDATION/TEST:")
print("=" * 40)
total_val = 0
for class_name, count in sorted(val_counts.items()):
    print(f"   {class_name}: {count} ảnh")
    total_val += count
print(f"   ─────────────────────────")
print(f"   Tổng cộng: {total_val} ảnh")

print(f"
📈 TỔNG SỐ ẢNH TOÀN BỘ DATASET: {total_train + total_val} ảnh")

In [ ]:
# ============================================
# BIỂU ĐỒ THANH - PHÂN BỐ ẢNH THEO LỚP
# ============================================

# Tạo figure với 2 subplot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Biểu đồ 1: Tập Train
if train_counts:
    classes = list(train_counts.keys())
    counts = list(train_counts.values())
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']  # Đỏ, Xanh lá, Xanh dương
    
    bars1 = axes[0].bar(classes, counts, color=colors[:len(classes)], edgecolor='black', linewidth=1.2)
    axes[0].set_title('📊 Phân Bố Ảnh - Tập Train', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Loại Bệnh', fontsize=12)
    axes[0].set_ylabel('Số Lượng Ảnh', fontsize=12)
    axes[0].tick_params(axis='x', rotation=15)
    
    # Thêm nhãn số lượng trên mỗi cột
    for bar, count in zip(bars1, counts):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                     str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')

# Biểu đồ 2: Tập Validation/Test
if val_counts:
    classes = list(val_counts.keys())
    counts = list(val_counts.values())
    
    bars2 = axes[1].bar(classes, counts, color=colors[:len(classes)], edgecolor='black', linewidth=1.2)
    axes[1].set_title('📊 Phân Bố Ảnh - Tập Validation/Test', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Loại Bệnh', fontsize=12)
    axes[1].set_ylabel('Số Lượng Ảnh', fontsize=12)
    axes[1].tick_params(axis='x', rotation=15)
    
    # Thêm nhãn số lượng trên mỗi cột
    for bar, count in zip(bars2, counts):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                     str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('distribution_by_class.png', dpi=150, bbox_inches='tight')
plt.show()

print("
💾 Biểu đồ đã được lưu: distribution_by_class.png")

In [ ]:
# ============================================
# BIỂU ĐỒ TRÒN - TỈ LỆ PHẦN TRĂM TỪNG LỚP
# ============================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
explode = (0.05, 0.05, 0.05)  # Tách nhẹ các phần

# Biểu đồ tròn tập Train
if train_counts:
    labels = list(train_counts.keys())
    sizes = list(train_counts.values())
    
    wedges, texts, autotexts = axes[0].pie(
        sizes, explode=explode[:len(sizes)], labels=labels, colors=colors[:len(sizes)],
        autopct='%1.1f%%', shadow=True, startangle=90,
        textprops={'fontsize': 11, 'fontweight': 'bold'}
    )
    axes[0].set_title('🥧 Tỉ Lệ Các Lớp - Tập Train', fontsize=14, fontweight='bold')

# Biểu đồ tròn tập Validation
if val_counts:
    labels = list(val_counts.keys())
    sizes = list(val_counts.values())
    
    wedges, texts, autotexts = axes[1].pie(
        sizes, explode=explode[:len(sizes)], labels=labels, colors=colors[:len(sizes)],
        autopct='%1.1f%%', shadow=True, startangle=90,
        textprops={'fontsize': 11, 'fontweight': 'bold'}
    )
    axes[1].set_title('🥧 Tỉ Lệ Các Lớp - Tập Validation', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('pie_chart_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("
💾 Biểu đồ tròn đã được lưu: pie_chart_distribution.png")

In [ ]:
# ============================================
# HIỂN THỊ ẢNH MẪU TỪ MỖI LỚP
# ============================================

def display_sample_images(data_dir, num_samples=3):
    """Hiển thị ảnh mẫu từ mỗi lớp trong dataset"""
    if not os.path.exists(data_dir):
        print(f"❌ Không tìm thấy thư mục: {data_dir}")
        return
    
    classes = sorted([d for d in os.listdir(data_dir) 
                      if os.path.isdir(os.path.join(data_dir, d))])
    
    if not classes:
        print("❌ Không tìm thấy lớp nào trong thư mục")
        return
    
    fig, axes = plt.subplots(len(classes), num_samples, figsize=(4*num_samples, 4*len(classes)))
    
    # Đảm bảo axes là 2D array
    if len(classes) == 1:
        axes = [axes]
    
    class_labels = {
        'angular_leaf_spot': '🟡 Angular Leaf Spot (Vết Bệnh Góc)',
        'bean_rust': '🔴 Bean Rust (Bệnh Rỉ Sắt)',
        'healthy': '🟢 Healthy (Lá Khỏe)'
    }
    
    for i, class_name in enumerate(classes):
        class_path = os.path.join(data_dir, class_name)
        image_files = [f for f in os.listdir(class_path) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        # Chọn ngẫu nhiên các ảnh mẫu
        sample_files = random.sample(image_files, min(num_samples, len(image_files)))
        
        for j, img_file in enumerate(sample_files):
            img_path = os.path.join(class_path, img_file)
            img = Image.open(img_path)
            
            axes[i][j].imshow(img)
            axes[i][j].axis('off')
            
            # Tiêu đề cho ảnh đầu tiên của mỗi lớp
            if j == 0:
                label = class_labels.get(class_name, class_name)
                axes[i][j].set_title(f'{label}', fontsize=12, fontweight='bold', loc='left')
    
    plt.suptitle('🖼️ Ảnh Mẫu Từ Mỗi Lớp Bệnh', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('sample_images.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("
💾 Ảnh mẫu đã được lưu: sample_images.png")

# Hiển thị ảnh mẫu từ tập train
print("📸 ẢNH MẪU TỪ TẬP TRAIN:")
print("=" * 60)
display_sample_images(TRAIN_DIR, num_samples=4)

In [ ]:
# ============================================
# PHÂN TÍCH KÍCH THƯỚC ẢNH
# ============================================

def analyze_image_sizes(data_dir, sample_size=100):
    """Phân tích kích thước ảnh trong dataset"""
    widths = []
    heights = []
    aspect_ratios = []
    
    if not os.path.exists(data_dir):
        return widths, heights, aspect_ratios
    
    all_images = []
    for class_name in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    all_images.append(os.path.join(class_path, img_file))
    
    # Lấy mẫu ngẫu nhiên để phân tích
    sample_images = random.sample(all_images, min(sample_size, len(all_images)))
    
    for img_path in sample_images:
        try:
            with Image.open(img_path) as img:
                w, h = img.size
                widths.append(w)
                heights.append(h)
                aspect_ratios.append(w / h)
        except Exception as e:
            continue
    
    return widths, heights, aspect_ratios

# Phân tích kích thước ảnh
widths, heights, aspect_ratios = analyze_image_sizes(TRAIN_DIR, sample_size=200)

if widths:
    print("📐 PHÂN TÍCH KÍCH THƯỚC ẢNH:")
    print("=" * 50)
    print(f"   Số ảnh phân tích: {len(widths)}")
    print(f"
   📏 Chiều rộng (Width):")
    print(f"      Min: {min(widths)} px")
    print(f"      Max: {max(widths)} px")
    print(f"      Trung bình: {np.mean(widths):.1f} px")
    print(f"
   📏 Chiều cao (Height):")
    print(f"      Min: {min(heights)} px")
    print(f"      Max: {max(heights)} px")
    print(f"      Trung bình: {np.mean(heights):.1f} px")
    print(f"
   📐 Tỉ lệ khung hình (Aspect Ratio):")
    print(f"      Min: {min(aspect_ratios):.3f}")
    print(f"      Max: {max(aspect_ratios):.3f}")
    print(f"      Trung bình: {np.mean(aspect_ratios):.3f}")
    
    # Vẽ histogram kích thước
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].hist(widths, bins=20, color='#FF6B6B', edgecolor='black', alpha=0.7)
    axes[0].set_title('Phân Bố Chiều Rộng', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Width (px)')
    axes[0].set_ylabel('Số lượng ảnh')
    axes[0].axvline(np.mean(widths), color='red', linestyle='--', label=f'Mean: {np.mean(widths):.0f}')
    axes[0].legend()
    
    axes[1].hist(heights, bins=20, color='#4ECDC4', edgecolor='black', alpha=0.7)
    axes[1].set_title('Phân Bố Chiều Cao', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Height (px)')
    axes[1].set_ylabel('Số lượng ảnh')
    axes[1].axvline(np.mean(heights), color='red', linestyle='--', label=f'Mean: {np.mean(heights):.0f}')
    axes[1].legend()
    
    axes[2].hist(aspect_ratios, bins=20, color='#45B7D1', edgecolor='black', alpha=0.7)
    axes[2].set_title('Phân Bố Tỉ Lệ Khung Hình', fontsize=12, fontweight='bold')
    axes[2].set_xlabel('Aspect Ratio (W/H)')
    axes[2].set_ylabel('Số lượng ảnh')
    axes[2].axvline(np.mean(aspect_ratios), color='red', linestyle='--', label=f'Mean: {np.mean(aspect_ratios):.2f}')
    axes[2].legend()
    
    plt.tight_layout()
    plt.savefig('image_size_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("
💾 Biểu đồ phân bố kích thước đã được lưu: image_size_distribution.png")
else:
    print("❌ Không có dữ liệu ảnh để phân tích")

### 📝 Tổng Kết Khám Phá Dữ Liệu (EDA)

**Những điểm chính từ EDA:**

1. **Số lượng dữ liệu:** Dataset gồm khoảng ~1034 ảnh train và ~133 ảnh validation/test
2. **Phân bố lớp:** Dữ liệu tương đối cân bằng giữa 3 lớp
3. **Kích thước ảnh:** Ảnh có kích thước đa dạng, cần resize về kích thước chuẩn (224x224)
4. **Chất lượng ảnh:** Ảnh rõ ràng, dễ nhận diện đặc điểm bệnh

**Nhận xét:**
- Dataset có quy mô vừa phải, phù hợp cho việc thử nghiệm các mô hình Transfer Learning
- Cần sử dụng Data Augmentation để tăng số lượng ảnh và tránh overfitting
- Các lớp có sự khác biệt rõ ràng về màu sắc và hình dạng vết bệnh

---

## 3️⃣ Tiền Xử Lý Ảnh (Image Preprocessing)

Trong phần này, chúng ta sẽ:
- Resize ảnh về kích thước chuẩn (224×224)
- Chuẩn hóa giá trị pixel
- Chia tập Train/Validation
- Áp dụng Data Augmentation để tăng cường dữ liệu

In [ ]:
# ============================================
# THIẾT LẬP THAM SỐ TIỀN XỬ LÝ
# ============================================

# Kích thước ảnh đầu vào cho mô hình
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)

# Batch size - số ảnh xử lý cùng lúc
BATCH_SIZE = 32

# Số lớp phân loại
NUM_CLASSES = len(CLASSES) if CLASSES else 3

print("⚙️ THAM SỐ TIỀN XỬ LÝ:")
print("=" * 40)
print(f"   Kích thước ảnh: {IMG_WIDTH} x {IMG_HEIGHT}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Số lớp phân loại: {NUM_CLASSES}")
print(f"   Các lớp: {CLASSES}")

In [ ]:
# ============================================
# TẠO DATA GENERATORS VỚI AUGMENTATION
# ============================================

# Data Augmentation cho tập Train
# Các kỹ thuật augmentation giúp:
# - Tăng số lượng dữ liệu ảo
# - Giảm overfitting
# - Mô hình học được các đặc trưng bất biến

train_datagen = ImageDataGenerator(
    rescale=1./255,              # Chuẩn hóa pixel về [0, 1]
    rotation_range=40,           # Xoay ngẫu nhiên 0-40 độ
    width_shift_range=0.2,       # Dịch ngang 20%
    height_shift_range=0.2,      # Dịch dọc 20%
    shear_range=0.2,             # Cắt nghiêng
    zoom_range=0.2,              # Phóng to/thu nhỏ 20%
    horizontal_flip=True,        # Lật ngang
    vertical_flip=True,          # Lật dọc
    brightness_range=[0.8, 1.2], # Thay đổi độ sáng
    fill_mode='nearest'          # Điền pixel khi biến đổi
)

# Data Generator cho tập Validation (chỉ chuẩn hóa, không augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255
)

print("✅ Đã tạo Data Generators với các kỹ thuật Augmentation:")
print("   - Xoay ảnh (rotation)")
print("   - Dịch chuyển (shift)")
print("   - Cắt nghiêng (shear)")
print("   - Phóng to/thu nhỏ (zoom)")
print("   - Lật ngang/dọc (flip)")
print("   - Thay đổi độ sáng (brightness)")

In [ ]:
# ============================================
# TẢI DỮ LIỆU TỪ THƯ MỤC
# ============================================

# Tạo generator cho tập Train
print("📂 Đang tải dữ liệu train...")
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

# Tạo generator cho tập Validation
print("\n📂 Đang tải dữ liệu validation...")
val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Lưu mapping class
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

print("\n" + "=" * 50)
print("📊 THÔNG TIN DỮ LIỆU:")
print("=" * 50)
print(f"   Tập Train: {train_generator.samples} ảnh")
print(f"   Tập Validation: {val_generator.samples} ảnh")
print(f"   Số bước/epoch (train): {len(train_generator)}")
print(f"   Số bước/epoch (val): {len(val_generator)}")
print(f"\n   Class mapping: {class_indices}")

In [ ]:
# ============================================
# HIỂN THỊ ẢNH SAU KHI AUGMENTATION
# ============================================

def show_augmented_images(generator, num_images=8):
    """Hiển thị ảnh sau khi áp dụng augmentation"""
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    # Lấy một batch ảnh
    images, labels = next(generator)
    
    for i in range(min(num_images, len(images))):
        axes[i].imshow(images[i])
        axes[i].axis('off')
        
        # Tìm tên lớp từ one-hot encoding
        class_idx = np.argmax(labels[i])
        class_name = class_names[class_idx]
        axes[i].set_title(f'{class_name}', fontsize=11, fontweight='bold')
    
    plt.suptitle('🔄 Ảnh Sau Khi Data Augmentation', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('augmented_images.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n💾 Ảnh augmentation đã được lưu: augmented_images.png")

# Hiển thị ảnh sau augmentation
print("🔄 ẢNH SAU KHI ÁP DỤNG DATA AUGMENTATION:")
print("=" * 60)
show_augmented_images(train_generator)

# Reset generator
train_generator.reset()

---

## 4️⃣ Xây Dựng Mô Hình Deep Learning

Trong phần này, chúng ta sẽ xây dựng và huấn luyện **3 mô hình Transfer Learning** khác nhau:

1. **ResNet50** - Mô hình sâu với kết nối tắt (skip connections)
2. **MobileNetV2** - Mô hình nhẹ, tối ưu cho thiết bị di động
3. **VGG19** - Mô hình cổ điển với kiến trúc đơn giản

### Transfer Learning là gì?

Transfer Learning (Học chuyển giao) là kỹ thuật sử dụng mô hình đã được huấn luyện trên tập dữ liệu lớn (như ImageNet) và tinh chỉnh (fine-tune) cho bài toán cụ thể của mình.

**Ưu điểm:**
- Tiết kiệm thời gian huấn luyện
- Yêu cầu ít dữ liệu hơn
- Đạt độ chính xác cao hơn

In [ ]:
# ============================================
# THIẾT LẬP THAM SỐ HUẤN LUYỆN
# ============================================

# Số epoch huấn luyện
EPOCHS = 20

# Learning rate
LEARNING_RATE = 0.0001

# Early stopping patience
PATIENCE = 5

print("⚙️ THAM SỐ HUẤN LUYỆN:")
print("=" * 40)
print(f"   Số epoch: {EPOCHS}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Early stopping patience: {PATIENCE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Input shape: ({IMG_HEIGHT}, {IMG_WIDTH}, 3)")

In [ ]:
# ============================================
# THIẾT LẬP CALLBACKS
# ============================================

def get_callbacks(model_name):
    """Tạo callbacks cho quá trình huấn luyện"""
    
    # Early Stopping - dừng sớm khi không cải thiện
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    # Reduce Learning Rate - giảm learning rate khi không cải thiện
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
    
    # Model Checkpoint - lưu model tốt nhất
    checkpoint = ModelCheckpoint(
        f'best_{model_name}.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    
    return [early_stop, reduce_lr, checkpoint]

print("✅ Đã thiết lập callbacks:")
print("   - Early Stopping: Dừng khi val_loss không cải thiện sau 5 epoch")
print("   - Reduce LR: Giảm learning rate khi val_loss không cải thiện")
print("   - Model Checkpoint: Lưu model có val_accuracy cao nhất")

In [ ]:
# ============================================
# HÀM XÂY DỰNG MÔ HÌNH TRANSFER LEARNING
# ============================================

def build_transfer_model(base_model_class, model_name, input_shape=(224, 224, 3), num_classes=3):
    """
    Xây dựng mô hình Transfer Learning
    
    Parameters:
    - base_model_class: Class của mô hình cơ sở (ResNet50, MobileNetV2, VGG19)
    - model_name: Tên mô hình (để in thông tin)
    - input_shape: Kích thước ảnh đầu vào
    - num_classes: Số lớp phân loại
    
    Returns:
    - model: Mô hình đã được xây dựng
    """
    
    print(f"\n🔨 Đang xây dựng mô hình {model_name}...")
    print("=" * 50)
    
    # Tải mô hình pretrained (không bao gồm lớp fully connected cuối)
    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Đóng băng các layer của base model
    base_model.trainable = False
    
    # Xây dựng mô hình hoàn chỉnh
    model = models.Sequential([
        # Base model (pretrained)
        base_model,
        
        # Global Average Pooling
        layers.GlobalAveragePooling2D(),
        
        # Dense layers (fully connected)
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # Output layer
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"   ✅ Mô hình {model_name} đã được xây dựng!")
    print(f"   📊 Tổng số parameters: {model.count_params():,}")
    print(f"   📊 Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")
    
    return model

### 4.1 Mô hình ResNet50

**ResNet50** (Residual Network) là mô hình với 50 layers, sử dụng kết nối tắt (skip connections) để giải quyết vấn đề vanishing gradient trong mạng sâu.

**Đặc điểm:**
- 50 layers với 25.6M parameters
- Skip connections giúp training ổn định
- Phù hợp cho nhiều bài toán phân loại ảnh

In [ ]:
# ============================================
# XÂY DỰNG VÀ HUẤN LUYỆN MÔ HÌNH RESNET50
# ============================================

# Xây dựng mô hình ResNet50
model_resnet = build_transfer_model(
    ResNet50, 
    'ResNet50', 
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    num_classes=NUM_CLASSES
)

# Hiển thị kiến trúc mô hình
print("\n📋 KIẾN TRÚC MÔ HÌNH RESNET50:")
model_resnet.summary()

In [ ]:
# ============================================
# HUẤN LUYỆN MÔ HÌNH RESNET50
# ============================================

print("\n🚀 BẮT ĐẦU HUẤN LUYỆN RESNET50")
print("=" * 60)

# Reset generators
train_generator.reset()
val_generator.reset()

# Huấn luyện mô hình
history_resnet = model_resnet.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=get_callbacks('resnet50'),
    verbose=1
)

print("\n✅ Hoàn thành huấn luyện ResNet50!")

### 4.2 Mô hình MobileNetV2

**MobileNetV2** là mô hình nhẹ, được tối ưu cho các thiết bị có tài nguyên hạn chế (mobile, embedded devices).

**Đặc điểm:**
- Sử dụng Depthwise Separable Convolutions
- Inverted Residuals với Linear Bottlenecks
- Chỉ 3.4M parameters (nhẹ hơn nhiều so với ResNet50)
- Phù hợp cho ứng dụng thời gian thực

In [ ]:
# ============================================
# XÂY DỰNG VÀ HUẤN LUYỆN MÔ HÌNH MOBILENETV2
# ============================================

# Xây dựng mô hình MobileNetV2
model_mobilenet = build_transfer_model(
    MobileNetV2, 
    'MobileNetV2', 
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    num_classes=NUM_CLASSES
)

# Hiển thị kiến trúc mô hình
print("\n📋 KIẾN TRÚC MÔ HÌNH MOBILENETV2:")
model_mobilenet.summary()

In [ ]:
# ============================================
# HUẤN LUYỆN MÔ HÌNH MOBILENETV2
# ============================================

print("\n🚀 BẮT ĐẦU HUẤN LUYỆN MOBILENETV2")
print("=" * 60)

# Reset generators
train_generator.reset()
val_generator.reset()

# Huấn luyện mô hình
history_mobilenet = model_mobilenet.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=get_callbacks('mobilenetv2'),
    verbose=1
)

print("\n✅ Hoàn thành huấn luyện MobileNetV2!")

### 4.3 Mô hình VGG19

**VGG19** là mô hình cổ điển từ Visual Geometry Group (Oxford), với kiến trúc đơn giản nhưng hiệu quả.

**Đặc điểm:**
- 19 layers (16 convolutional + 3 fully connected)
- Sử dụng toàn bộ 3x3 convolutions
- 143.7M parameters (nặng nhất trong 3 mô hình)
- Đạt kết quả tốt trên nhiều bài toán phân loại

In [ ]:
# ============================================
# XÂY DỰNG VÀ HUẤN LUYỆN MÔ HÌNH VGG19
# ============================================

# Xây dựng mô hình VGG19
model_vgg = build_transfer_model(
    VGG19, 
    'VGG19', 
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    num_classes=NUM_CLASSES
)

# Hiển thị kiến trúc mô hình
print("\n📋 KIẾN TRÚC MÔ HÌNH VGG19:")
model_vgg.summary()

In [ ]:
# ============================================
# HUẤN LUYỆN MÔ HÌNH VGG19
# ============================================

print("\n🚀 BẮT ĐẦU HUẤN LUYỆN VGG19")
print("=" * 60)

# Reset generators
train_generator.reset()
val_generator.reset()

# Huấn luyện mô hình
history_vgg = model_vgg.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=get_callbacks('vgg19'),
    verbose=1
)

print("\n✅ Hoàn thành huấn luyện VGG19!")

---

## 5️⃣ Đánh Giá & Phân Tích Kết Quả

Trong phần này, chúng ta sẽ:
- Vẽ Learning Curves (Loss và Accuracy theo epoch)
- Tính các metrics: Accuracy, Precision, Recall, F1-score
- Hiển thị Confusion Matrix
- So sánh hiệu suất của các mô hình

In [ ]:
# ============================================
# HÀM VẼ LEARNING CURVES
# ============================================

def plot_learning_curves(history, model_name):
    """
    Vẽ biểu đồ Learning Curves cho một mô hình
    
    Parameters:
    - history: Lịch sử huấn luyện từ model.fit()
    - model_name: Tên mô hình (để hiển thị tiêu đề)
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Lấy dữ liệu từ history
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(1, len(acc) + 1)
    
    # Biểu đồ Accuracy
    axes[0].plot(epochs_range, acc, 'b-', label='Training Accuracy', linewidth=2)
    axes[0].plot(epochs_range, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
    axes[0].set_title(f'{model_name} - Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].legend(loc='lower right', fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([0, 1])
    
    # Biểu đồ Loss
    axes[1].plot(epochs_range, loss, 'b-', label='Training Loss', linewidth=2)
    axes[1].plot(epochs_range, val_loss, 'r-', label='Validation Loss', linewidth=2)
    axes[1].set_title(f'{model_name} - Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].legend(loc='upper right', fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'learning_curves_{model_name.lower().replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # In kết quả cuối cùng
    print(f"\n📊 KẾT QUẢ HUẤN LUYỆN {model_name.upper()}:")
    print("=" * 50)
    print(f"   Training Accuracy (cuối): {acc[-1]:.4f}")
    print(f"   Validation Accuracy (cuối): {val_acc[-1]:.4f}")
    print(f"   Training Loss (cuối): {loss[-1]:.4f}")
    print(f"   Validation Loss (cuối): {val_loss[-1]:.4f}")
    print(f"   Best Validation Accuracy: {max(val_acc):.4f} (Epoch {val_acc.index(max(val_acc)) + 1})")

print("✅ Hàm vẽ Learning Curves đã được định nghĩa!")

In [ ]:
# ============================================
# VẼ LEARNING CURVES CHO TẤT CẢ MÔ HÌNH
# ============================================

print("📈 LEARNING CURVES - RESNET50")
print("=" * 60)
plot_learning_curves(history_resnet, 'ResNet50')

print("\n" + "=" * 60)
print("📈 LEARNING CURVES - MOBILENETV2")
print("=" * 60)
plot_learning_curves(history_mobilenet, 'MobileNetV2')

print("\n" + "=" * 60)
print("📈 LEARNING CURVES - VGG19")
print("=" * 60)
plot_learning_curves(history_vgg, 'VGG19')

In [ ]:
# ============================================
# HÀM ĐÁNH GIÁ MÔ HÌNH
# ============================================

def evaluate_model(model, generator, model_name, class_names):
    """
    Đánh giá mô hình và hiển thị các metrics
    
    Parameters:
    - model: Mô hình đã huấn luyện
    - generator: Data generator cho tập test/validation
    - model_name: Tên mô hình
    - class_names: Danh sách tên các lớp
    
    Returns:
    - metrics_dict: Dictionary chứa các metrics
    """
    print(f"\n📊 ĐÁNH GIÁ MÔ HÌNH: {model_name}")
    print("=" * 60)
    
    # Reset generator và lấy tất cả dữ liệu
    generator.reset()
    
    # Dự đoán
    predictions = model.predict(generator, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    y_true = generator.classes
    
    # Tính các metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    # In kết quả
    print(f"\n📈 METRICS TỔNG QUAN:")
    print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    
    # Classification Report chi tiết
    print(f"\n📋 CLASSIFICATION REPORT:")
    print("-" * 60)
    report = classification_report(y_true, y_pred, target_names=class_names)
    print(report)
    
    # Trả về metrics
    metrics_dict = {
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'y_true': y_true,
        'y_pred': y_pred
    }
    
    return metrics_dict

print("✅ Hàm đánh giá mô hình đã được định nghĩa!")

In [ ]:
# ============================================
# HÀM VẼ CONFUSION MATRIX
# ============================================

def plot_confusion_matrix(y_true, y_pred, class_names, model_name):
    """
    Vẽ Confusion Matrix với heatmap
    
    Parameters:
    - y_true: Nhãn thực tế
    - y_pred: Nhãn dự đoán
    - class_names: Danh sách tên các lớp
    - model_name: Tên mô hình
    """
    # Tính confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Vẽ heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names,
        annot_kws={'size': 14, 'fontweight': 'bold'}
    )
    
    plt.title(f'Confusion Matrix - {model_name}', fontsize=16, fontweight='bold')
    plt.xlabel('Dự Đoán (Predicted)', fontsize=12)
    plt.ylabel('Thực Tế (Actual)', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{model_name.lower().replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Phân tích confusion matrix
    print(f"\n📊 PHÂN TÍCH CONFUSION MATRIX - {model_name}:")
    print("=" * 50)
    total = np.sum(cm)
    for i, class_name in enumerate(class_names):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = total - tp - fn - fp
        
        class_accuracy = tp / np.sum(cm[i, :]) if np.sum(cm[i, :]) > 0 else 0
        print(f"   {class_name}:")
        print(f"      - Dự đoán đúng: {tp}/{np.sum(cm[i, :])} ({class_accuracy*100:.1f}%)")
        print(f"      - Dự đoán sai: {fn}/{np.sum(cm[i, :])} ({(1-class_accuracy)*100:.1f}%)")

print("✅ Hàm vẽ Confusion Matrix đã được định nghĩa!")

In [ ]:
# ============================================
# ĐÁNH GIÁ TẤT CẢ CÁC MÔ HÌNH
# ============================================

# Đánh giá ResNet50
print("\n" + "=" * 70)
print("🔵 ĐÁNH GIÁ MÔ HÌNH RESNET50")
print("=" * 70)
metrics_resnet = evaluate_model(model_resnet, val_generator, 'ResNet50', class_names)
plot_confusion_matrix(metrics_resnet['y_true'], metrics_resnet['y_pred'], class_names, 'ResNet50')

# Đánh giá MobileNetV2
print("\n" + "=" * 70)
print("🟢 ĐÁNH GIÁ MÔ HÌNH MOBILENETV2")
print("=" * 70)
metrics_mobilenet = evaluate_model(model_mobilenet, val_generator, 'MobileNetV2', class_names)
plot_confusion_matrix(metrics_mobilenet['y_true'], metrics_mobilenet['y_pred'], class_names, 'MobileNetV2')

# Đánh giá VGG19
print("\n" + "=" * 70)
print("🟣 ĐÁNH GIÁ MÔ HÌNH VGG19")
print("=" * 70)
metrics_vgg = evaluate_model(model_vgg, val_generator, 'VGG19', class_names)
plot_confusion_matrix(metrics_vgg['y_true'], metrics_vgg['y_pred'], class_names, 'VGG19')

---

## 6️⃣ So Sánh Mô Hình và Kết Luận

Trong phần này, chúng ta sẽ:
- So sánh hiệu suất của 3 mô hình
- Phân tích ưu nhược điểm của từng mô hình
- Đưa ra kết luận và đề xuất mô hình tốt nhất

In [ ]:
# ============================================
# SO SÁNH HIỆU SUẤT CÁC MÔ HÌNH
# ============================================

# Tạo DataFrame so sánh
comparison_data = {
    'Mô Hình': ['ResNet50', 'MobileNetV2', 'VGG19'],
    'Accuracy': [
        metrics_resnet['accuracy'],
        metrics_mobilenet['accuracy'],
        metrics_vgg['accuracy']
    ],
    'Precision': [
        metrics_resnet['precision'],
        metrics_mobilenet['precision'],
        metrics_vgg['precision']
    ],
    'Recall': [
        metrics_resnet['recall'],
        metrics_mobilenet['recall'],
        metrics_vgg['recall']
    ],
    'F1-Score': [
        metrics_resnet['f1_score'],
        metrics_mobilenet['f1_score'],
        metrics_vgg['f1_score']
    ]
}

comparison_df = pd.DataFrame(comparison_data)

# Định dạng bảng
print("\n📊 BẢNG SO SÁNH HIỆU SUẤT CÁC MÔ HÌNH")
print("=" * 70)
print(comparison_df.to_string(index=False, float_format='{:.4f}'.format))

# Tìm mô hình tốt nhất
best_model_idx = comparison_df['Accuracy'].idxmax()
best_model = comparison_df.loc[best_model_idx, 'Mô Hình']
best_accuracy = comparison_df.loc[best_model_idx, 'Accuracy']

print(f"\n🏆 MÔ HÌNH TỐT NHẤT: {best_model}")
print(f"   Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")

In [ ]:
# ============================================
# BIỂU ĐỒ SO SÁNH CÁC MÔ HÌNH
# ============================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Biểu đồ cột so sánh các metrics
x = np.arange(len(comparison_df['Mô Hình']))
width = 0.2

bars1 = axes[0].bar(x - 1.5*width, comparison_df['Accuracy'], width, label='Accuracy', color='#FF6B6B')
bars2 = axes[0].bar(x - 0.5*width, comparison_df['Precision'], width, label='Precision', color='#4ECDC4')
bars3 = axes[0].bar(x + 0.5*width, comparison_df['Recall'], width, label='Recall', color='#45B7D1')
bars4 = axes[0].bar(x + 1.5*width, comparison_df['F1-Score'], width, label='F1-Score', color='#96CEB4')

axes[0].set_xlabel('Mô Hình', fontsize=12)
axes[0].set_ylabel('Score', fontsize=12)
axes[0].set_title('So Sánh Các Metrics Giữa Các Mô Hình', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(comparison_df['Mô Hình'])
axes[0].legend()
axes[0].set_ylim([0, 1.1])
axes[0].grid(axis='y', alpha=0.3)

# Thêm giá trị trên cột
for bars in [bars1, bars2, bars3, bars4]:
    for bar in bars:
        height = bar.get_height()
        axes[0].annotate(f'{height:.2f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=8)

# Biểu đồ radar
categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
N = len(categories)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Đóng vòng tròn

ax_radar = plt.subplot(1, 2, 2, projection='polar')

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
for i, model_name in enumerate(comparison_df['Mô Hình']):
    values = comparison_df.loc[i, ['Accuracy', 'Precision', 'Recall', 'F1-Score']].values.tolist()
    values += values[:1]  # Đóng vòng tròn
    ax_radar.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[i])
    ax_radar.fill(angles, values, alpha=0.25, color=colors[i])

ax_radar.set_xticks(angles[:-1])
ax_radar.set_xticklabels(categories, fontsize=10)
ax_radar.set_title('Biểu Đồ Radar So Sánh', fontsize=14, fontweight='bold', y=1.08)
ax_radar.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 Biểu đồ so sánh đã được lưu: model_comparison.png")

In [ ]:
# ============================================
# PHÂN TÍCH KẾT QUẢ VÀ KẾT LUẬN
# ============================================

print("📝 PHÂN TÍCH KẾT QUẢ VÀ KẾT LUẬN")
print("=" * 70)

# Phân tích từng mô hình
models_analysis = {
    'ResNet50': {
        'pros': [
            'Kiến trúc sâu với skip connections giúp học được đặc trưng phức tạp',
            'Ổn định khi training với nhiều layer',
            'Hiệu suất tốt trên nhiều bài toán phân loại ảnh'
        ],
        'cons': [
            'Yêu cầu nhiều tài nguyên tính toán',
            'Thời gian inference lâu hơn MobileNet'
        ]
    },
    'MobileNetV2': {
        'pros': [
            'Nhẹ, nhanh, phù hợp cho thiết bị di động',
            'Depthwise Separable Convolutions giảm số parameters',
            'Thời gian training và inference nhanh'
        ],
        'cons': [
            'Có thể không đạt accuracy cao nhất với dữ liệu phức tạp',
            'Trade-off giữa tốc độ và độ chính xác'
        ]
    },
    'VGG19': {
        'pros': [
            'Kiến trúc đơn giản, dễ hiểu và implement',
            'Đạt kết quả tốt trên nhiều benchmark',
            'Pretrained weights chất lượng cao từ ImageNet'
        ],
        'cons': [
            'Nhiều parameters nhất (nặng nhất)',
            'Tốn nhiều bộ nhớ và thời gian tính toán'
        ]
    }
}

for model_name, analysis in models_analysis.items():
    print(f"\n🔹 {model_name}:")
    print("   Ưu điểm:")
    for pro in analysis['pros']:
        print(f"      ✅ {pro}")
    print("   Nhược điểm:")
    for con in analysis['cons']:
        print(f"      ⚠️ {con}")

# Kết luận
print("\n" + "=" * 70)
print("🎯 KẾT LUẬN:")
print("=" * 70)
print(f"""
1. MÔ HÌNH TỐT NHẤT: {best_model} với Accuracy: {best_accuracy*100:.2f}%

2. SO SÁNH:
   - ResNet50: Phù hợp khi cần độ chính xác cao, có GPU mạnh
   - MobileNetV2: Phù hợp cho ứng dụng thời gian thực, mobile
   - VGG19: Baseline tốt, kiến trúc cổ điển đã được kiểm chứng

3. ĐỀ XUẤT:
   - Với ứng dụng thực tế: Sử dụng {best_model} hoặc MobileNetV2
   - Với nghiên cứu: So sánh thêm với EfficientNet, Vision Transformer
   - Để tăng accuracy: Fine-tune thêm layers, tăng data augmentation
""")

---

## 7️⃣ Đề Xuất Nâng Cao và Hướng Phát Triển

### 7.1 Các kỹ thuật nâng cao có thể áp dụng

1. **Grad-CAM (Gradient-weighted Class Activation Mapping)**
   - Trực quan hóa vùng ảnh mà mô hình tập trung khi đưa ra dự đoán
   - Giúp giải thích và debug mô hình

2. **EfficientNet**
   - Mô hình hiện đại với hiệu suất state-of-the-art
   - Cân bằng giữa độ sâu, chiều rộng và độ phân giải

3. **Vision Transformer (ViT)**
   - Áp dụng kiến trúc Transformer cho Computer Vision
   - Hiệu quả với dữ liệu lớn

4. **Data Augmentation nâng cao**
   - MixUp, CutMix, AutoAugment
   - Tăng cường dữ liệu thông minh hơn

5. **Ensemble Learning**
   - Kết hợp nhiều mô hình để tăng độ chính xác
   - Voting hoặc stacking

In [ ]:
# ============================================
# GRAD-CAM - GIẢI THÍCH DỰ ĐOÁN CỦA MÔ HÌNH
# ============================================

def make_gradcam_heatmap(model, img_array, pred_index=None):
    """
    Tạo Grad-CAM heatmap để giải thích dự đoán của mô hình
    
    Parameters:
    - model: Mô hình đã huấn luyện
    - img_array: Ảnh đầu vào (đã tiền xử lý)
    - pred_index: Index của class muốn visualize (None = class có xác suất cao nhất)
    
    Returns:
    - heatmap: Grad-CAM heatmap
    """
    # Lấy layer convolution cuối cùng
    # Tìm layer conv cuối trong base model
    last_conv_layer = None
    for layer in reversed(model.layers):
        if 'conv' in layer.name.lower() or hasattr(layer, 'layers'):
            if hasattr(layer, 'layers'):
                for sub_layer in reversed(layer.layers):
                    if 'conv' in sub_layer.name.lower():
                        last_conv_layer_name = sub_layer.name
                        last_conv_layer = layer.get_layer(last_conv_layer_name)
                        break
            else:
                last_conv_layer = layer
            if last_conv_layer is not None:
                break
    
    if last_conv_layer is None:
        print("Không tìm thấy layer convolution!")
        return None
    
    # Tạo model để lấy gradient
    grad_model = tf.keras.Model(
        model.inputs,
        [model.get_layer(model.layers[0].name).output, model.output]
    )
    
    # Tính gradient
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    
    # Gradient của output class theo feature map
    grads = tape.gradient(class_channel, conv_outputs)
    
    if grads is None:
        return None
    
    # Global average pooling của gradient
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight feature map với gradient
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    # Normalize heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

print("✅ Hàm Grad-CAM đã được định nghĩa!")
print("\nGrad-CAM giúp trực quan hóa vùng ảnh mà mô hình tập trung khi đưa ra dự đoán.")
print("Điều này giúp chúng ta hiểu và tin tưởng hơn vào quyết định của mô hình.")

### 7.2 Hướng phát triển tiếp theo

📌 **Cải thiện mô hình:**
- Thử nghiệm với EfficientNet-B4, B5 (có thể đạt >95% accuracy)
- Áp dụng Vision Transformer nếu có đủ dữ liệu
- Sử dụng Cross-Validation để đánh giá ổn định hơn

📌 **Mở rộng ứng dụng:**
- Xây dựng API REST với Flask/FastAPI để deploy mô hình
- Tạo ứng dụng mobile với TensorFlow Lite
- Phát triển giao diện web để người dùng upload ảnh và nhận kết quả

📌 **Tích hợp với các dataset khác:**
- PlantVillage Dataset: Dataset lớn hơn với nhiều loại bệnh cây
- Kết hợp với dữ liệu thực tế thu thập từ nông trại

📌 **Tài nguyên tham khảo:**
- [GitHub: Bean Leaf Classification](https://github.com/topics/bean-leaf-classification)
- [Kaggle: Plant Disease Classification](https://www.kaggle.com/datasets)
- [TensorFlow: Image Classification Tutorial](https://www.tensorflow.org/tutorials/images/classification)

In [ ]:
# ============================================
# LƯU CÁC MÔ HÌNH ĐÃ HUẤN LUYỆN
# ============================================

# Lưu các mô hình
print("💾 LƯU CÁC MÔ HÌNH ĐÃ HUẤN LUYỆN")
print("=" * 50)

try:
    model_resnet.save('model_resnet50.keras')
    print("✅ Đã lưu: model_resnet50.keras")
except Exception as e:
    print(f"⚠️ Lỗi lưu ResNet50: {e}")

try:
    model_mobilenet.save('model_mobilenetv2.keras')
    print("✅ Đã lưu: model_mobilenetv2.keras")
except Exception as e:
    print(f"⚠️ Lỗi lưu MobileNetV2: {e}")

try:
    model_vgg.save('model_vgg19.keras')
    print("✅ Đã lưu: model_vgg19.keras")
except Exception as e:
    print(f"⚠️ Lỗi lưu VGG19: {e}")

print("\n📁 Các mô hình đã được lưu, có thể load lại bằng:")
print("   model = tf.keras.models.load_model('model_xxx.keras')")

### 7.3 Demo Dự Đoán Trên Ảnh Mới

Phần này minh họa cách sử dụng mô hình đã huấn luyện để dự đoán trên ảnh mới.

In [ ]:
# ============================================
# DEMO DỰ ĐOÁN TRÊN ẢNH MỚI
# ============================================

def predict_single_image(model, image_path, class_names, img_size=(224, 224)):
    """
    Dự đoán lớp bệnh cho một ảnh lá đậu
    
    Parameters:
    - model: Mô hình đã huấn luyện
    - image_path: Đường dẫn đến ảnh
    - class_names: Danh sách tên các lớp
    - img_size: Kích thước ảnh đầu vào
    
    Returns:
    - predicted_class: Tên lớp được dự đoán
    - confidence: Độ tin cậy của dự đoán
    """
    # Đọc và tiền xử lý ảnh
    img = Image.open(image_path)
    img_resized = img.resize(img_size)
    img_array = np.array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Dự đoán
    predictions = model.predict(img_array, verbose=0)
    predicted_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_idx]
    predicted_class = class_names[predicted_idx]
    
    return predicted_class, confidence, predictions[0]

def visualize_prediction(image_path, predicted_class, confidence, all_probs, class_names):
    """Trực quan hóa kết quả dự đoán"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Hiển thị ảnh
    img = Image.open(image_path)
    axes[0].imshow(img)
    axes[0].axis('off')
    
    # Màu sắc theo loại bệnh
    color_map = {
        'healthy': '#4CAF50',
        'angular_leaf_spot': '#FF9800', 
        'bean_rust': '#F44336'
    }
    title_color = color_map.get(predicted_class.lower(), '#2196F3')
    
    axes[0].set_title(f'Dự đoán: {predicted_class}\nĐộ tin cậy: {confidence*100:.1f}%', 
                      fontsize=14, fontweight='bold', color=title_color)
    
    # Biểu đồ xác suất
    colors = [color_map.get(c.lower(), '#2196F3') for c in class_names]
    bars = axes[1].barh(class_names, all_probs, color=colors, edgecolor='black')
    axes[1].set_xlabel('Xác suất', fontsize=12)
    axes[1].set_title('Phân bố xác suất các lớp', fontsize=14, fontweight='bold')
    axes[1].set_xlim([0, 1])
    
    # Thêm nhãn giá trị
    for bar, prob in zip(bars, all_probs):
        axes[1].text(prob + 0.02, bar.get_y() + bar.get_height()/2, 
                    f'{prob*100:.1f}%', va='center', fontsize=11)
    
    plt.tight_layout()
    plt.savefig('prediction_demo.png', dpi=150, bbox_inches='tight')
    plt.show()

print("✅ Hàm demo dự đoán đã được định nghĩa!")
print("\n📌 Cách sử dụng:")
print("   predicted_class, confidence, probs = predict_single_image(model_resnet, 'path/to/image.jpg', class_names)")
print("   visualize_prediction('path/to/image.jpg', predicted_class, confidence, probs, class_names)")

In [ ]:
# ============================================
# DEMO: DỰ ĐOÁN TRÊN MỘT SỐ ẢNH TỪ TẬP VALIDATION
# ============================================

def demo_predictions(model, model_name, data_dir, class_names, num_samples=6):
    """Demo dự đoán trên nhiều ảnh mẫu"""
    
    print(f"\n�� DEMO DỰ ĐOÁN VỚI MÔ HÌNH {model_name}")
    print("=" * 60)
    
    # Lấy ngẫu nhiên một số ảnh từ mỗi lớp
    sample_images = []
    for class_name in class_names:
        class_path = os.path.join(data_dir, class_name)
        if os.path.exists(class_path):
            images = [f for f in os.listdir(class_path) 
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                selected = random.sample(images, min(num_samples // len(class_names), len(images)))
                for img in selected:
                    sample_images.append((os.path.join(class_path, img), class_name))
    
    # Dự đoán và hiển thị
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    correct = 0
    for i, (img_path, true_label) in enumerate(sample_images[:6]):
        pred_class, confidence, probs = predict_single_image(model, img_path, class_names)
        
        # Hiển thị ảnh
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')
        
        # Kiểm tra đúng/sai
        is_correct = pred_class == true_label
        if is_correct:
            correct += 1
            border_color = 'green'
            result_text = '✓'
        else:
            border_color = 'red'
            result_text = '✗'
        
        title = f"Thực tế: {true_label}\nDự đoán: {pred_class} ({confidence*100:.1f}%) {result_text}"
        axes[i].set_title(title, fontsize=10, color=border_color, fontweight='bold')
        
        # Thêm border
        for spine in axes[i].spines.values():
            spine.set_edgecolor(border_color)
            spine.set_linewidth(3)
            spine.set_visible(True)
    
    plt.suptitle(f'Demo Dự Đoán - {model_name} ({correct}/{len(sample_images[:6])} đúng)', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'demo_predictions_{model_name.lower()}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 Kết quả demo: {correct}/{len(sample_images[:6])} dự đoán đúng")

# Chạy demo với mô hình tốt nhất
print("🎯 Chạy demo dự đoán với các mô hình đã huấn luyện...")
demo_predictions(model_resnet, 'ResNet50', VAL_DIR, class_names)

### 7.4 Xuất Mô Hình và Triển Khai

Phần này hướng dẫn cách xuất mô hình để triển khai trong ứng dụng thực tế.

In [ ]:
# ============================================
# XUẤT MÔ HÌNH CHO TRIỂN KHAI
# ============================================

def export_model_for_deployment(model, model_name, export_dir='exported_models'):
    """
    Xuất mô hình ở nhiều định dạng cho triển khai
    
    Parameters:
    - model: Mô hình đã huấn luyện
    - model_name: Tên mô hình
    - export_dir: Thư mục xuất
    """
    import os
    
    os.makedirs(export_dir, exist_ok=True)
    
    print(f"\n💾 XUẤT MÔ HÌNH: {model_name}")
    print("=" * 50)
    
    # 1. Lưu định dạng Keras (.keras)
    keras_path = os.path.join(export_dir, f'{model_name}.keras')
    model.save(keras_path)
    print(f"✅ Đã lưu định dạng Keras: {keras_path}")
    
    # 2. Lưu định dạng H5 (.h5) - tương thích với nhiều phiên bản
    h5_path = os.path.join(export_dir, f'{model_name}.h5')
    model.save(h5_path)
    print(f"✅ Đã lưu định dạng H5: {h5_path}")
    
    # 3. Lưu định dạng SavedModel (cho TensorFlow Serving)
    savedmodel_path = os.path.join(export_dir, f'{model_name}_savedmodel')
    model.save(savedmodel_path, save_format='tf')
    print(f"✅ Đã lưu định dạng SavedModel: {savedmodel_path}")
    
    # 4. Chuyển đổi sang TFLite (cho mobile)
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        tflite_path = os.path.join(export_dir, f'{model_name}.tflite')
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"✅ Đã lưu định dạng TFLite: {tflite_path}")
    except Exception as e:
        print(f"⚠️ Không thể chuyển đổi TFLite: {e}")
    
    print(f"\n📁 Các file đã xuất trong thư mục: {export_dir}/")
    return export_dir

# Xuất mô hình tốt nhất
print("📦 Xuất mô hình để triển khai...")
# Bỏ comment dòng dưới để xuất mô hình
# export_model_for_deployment(model_resnet, 'resnet50_bean_leaf')

In [ ]:
# ============================================
# TỔNG KẾT THỐNG KÊ
# ============================================

print("📊 TỔNG KẾT THỐNG KÊ TOÀN BỘ NOTEBOOK")
print("=" * 70)

# Thông tin dataset
print("\n📁 THÔNG TIN DATASET:")
print(f"   Tổng số ảnh train: {train_generator.samples}")
print(f"   Tổng số ảnh validation: {val_generator.samples}")
print(f"   Số lớp phân loại: {NUM_CLASSES}")
print(f"   Kích thước ảnh: {IMG_WIDTH}x{IMG_HEIGHT}")

# Thông tin huấn luyện
print("\n⚙️ CẤU HÌNH HUẤN LUYỆN:")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Số epoch: {EPOCHS}")
print(f"   Learning rate: {LEARNING_RATE}")

# Bảng tổng kết các mô hình
print("\n🏆 KẾT QUẢ CÁC MÔ HÌNH:")
print("-" * 70)
print(f"{'Mô hình':<15} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 70)

models_results = [
    ('ResNet50', metrics_resnet),
    ('MobileNetV2', metrics_mobilenet),
    ('VGG19', metrics_vgg)
]

best_acc = 0
best_model_name = ''
for name, metrics in models_results:
    print(f"{name:<15} {metrics['accuracy']:<12.4f} {metrics['precision']:<12.4f} {metrics['recall']:<12.4f} {metrics['f1_score']:<12.4f}")
    if metrics['accuracy'] > best_acc:
        best_acc = metrics['accuracy']
        best_model_name = name

print("-" * 70)
print(f"\n🥇 MÔ HÌNH TỐT NHẤT: {best_model_name} với Accuracy = {best_acc*100:.2f}%")

# Lưu kết quả vào file CSV
results_df = pd.DataFrame({
    'Model': [name for name, _ in models_results],
    'Accuracy': [m['accuracy'] for _, m in models_results],
    'Precision': [m['precision'] for _, m in models_results],
    'Recall': [m['recall'] for _, m in models_results],
    'F1-Score': [m['f1_score'] for _, m in models_results]
})
results_df.to_csv('model_results.csv', index=False)
print("\n💾 Kết quả đã được lưu vào: model_results.csv")

---

## 📋 Tổng Kết

### Những gì đã thực hiện trong notebook này:

1. ✅ **Nhập thư viện và tải dữ liệu** - Sử dụng TensorFlow/Keras và các thư viện hỗ trợ

2. ✅ **Khám phá dữ liệu (EDA)** - Phân tích số lượng ảnh, kích thước, phân bố các lớp với biểu đồ trực quan

3. ✅ **Tiền xử lý ảnh** - Resize, chuẩn hóa, Data Augmentation để tăng cường dữ liệu

4. ✅ **Xây dựng mô hình Deep Learning** - Transfer Learning với 3 mô hình:
   - ResNet50
   - MobileNetV2
   - VGG19

5. ✅ **Đánh giá và phân tích kết quả** - Accuracy, Precision, Recall, F1-Score, Confusion Matrix

6. ✅ **So sánh mô hình** - Bảng so sánh và biểu đồ radar

7. ✅ **Đề xuất nâng cao** - Grad-CAM, EfficientNet, Vision Transformer, hướng phát triển

---

### 📚 Tài liệu tham khảo

1. [Kaggle Dataset: Bean Leaf Lesions Classification](https://www.kaggle.com/datasets/marquis03/bean-leaf-lesions-classification)
2. [TensorFlow Transfer Learning Guide](https://www.tensorflow.org/tutorials/images/transfer_learning)
3. [Keras Applications](https://keras.io/api/applications/)
4. [GitHub: Plant Disease Classification Projects](https://github.com/topics/plant-disease-classification)

---

### 👨‍💻 Thông tin

- **Ngôn ngữ:** Python 3.10+
- **Framework:** TensorFlow/Keras
- **Mục đích:** Bài tập lớn môn Khai Phá Dữ Liệu

---

**© 2024 - Bean Leaf Lesions Classification Project**

In [ ]:
# ============================================
# THÔNG TIN CÀI ĐẶT
# ============================================

print("📦 CÁC THƯ VIỆN CẦN CÀI ĐẶT:")
print("=" * 50)
print("""
# File: requirements.txt

# Core Data Science
numpy>=1.21.0
pandas>=1.3.0

# Visualization
matplotlib>=3.4.0
seaborn>=0.11.0

# Deep Learning
tensorflow>=2.10.0

# Image Processing
Pillow>=8.0.0

# Machine Learning Utilities
scikit-learn>=0.24.0

# Kaggle API (để tải dataset)
kaggle>=1.5.12
""")

print("\n💻 CÁCH CÀI ĐẶT:")
print("   pip install -r requirements.txt")

print("\n🚀 CÁCH CHẠY NOTEBOOK:")
print("   1. Cài đặt các thư viện: pip install -r requirements.txt")
print("   2. Tải dataset: python download_dataset.py")
print("   3. Mở notebook: jupyter notebook bean_leaf_classification.ipynb")
print("   4. Chạy từng cell theo thứ tự")